In [15]:
import sympy as sp
import numpy as np
from scipy import linalg

## Linearization

In [2]:
lx , ly, alpha, theta = sp.symbols("lambda_x lambda_y alpha theta ")

In [3]:
l1 = lx * sp.cos(alpha)
l2 = lx * sp.sin(alpha + theta)
l3 = ly * sp.cos(alpha + theta)
l4 = -ly * sp.sin(alpha)

In [4]:
L = sp.Matrix([[l1, l2, l3, l4]]).T

In [5]:
L

Matrix([
[        lambda_x*cos(alpha)],
[lambda_x*sin(alpha + theta)],
[lambda_y*cos(alpha + theta)],
[       -lambda_y*sin(alpha)]])

In [6]:
L.jacobian((lx , ly, alpha, theta))

Matrix([
[        cos(alpha),                  0,         -lambda_x*sin(alpha),                            0],
[sin(alpha + theta),                  0,  lambda_x*cos(alpha + theta),  lambda_x*cos(alpha + theta)],
[                 0, cos(alpha + theta), -lambda_y*sin(alpha + theta), -lambda_y*sin(alpha + theta)],
[                 0,        -sin(alpha),         -lambda_y*cos(alpha),                            0]])

### Problem:
#### In triangle below , A, B and C have been observed, find the coordinates of B. AC is known

![q1.png](q1.png)

* $l_1 = 59^\circ 59' 58''$ 
* $l_2 = 59^\circ 59' 51''$ 
* $l_3 = 59^\circ 59' 54''$ 
* $AC = 1000 m$ 

## $l_1 = Tan^{-1}(\frac{y_B}{ x_B})$
## $l_3 = Tan^{-1}(\frac{y_B}{1000- x_B})$
## $l_2 = \pi - (Tan^{-1}(\frac{y_B}{ x_B}) + Tan^{-1}(\frac{y_B}{1000- x_B}))$

In [7]:
xb, yb, xc = sp.symbols("x_b y_b x_c")
L = sp.Matrix([[sp.atan(yb/ xb),
                sp.pi - (sp.atan(yb/ xb) +sp.atan(yb/(xc-xb))) 
                ,sp.atan(yb/(1000-xb))]]).T
var = sp.Matrix([xb, yb, xc])
l = sp.Matrix([[(59 + 59/60 + 58/3600) * sp.pi / 180],
              [(59 + 59/60 + 51/3600) * sp.pi / 180],
              [(59 + 59/60 + 54/3600) * sp.pi / 180]])
x = sp.Matrix([[1000/2, 500 * sp.tan(60 * sp.pi/180), 1000]]).T

### $x_{estimated} = (A^TA)^{-1}A^Tl$

In [8]:
for i in range(100):
    A1 = L.jacobian(var)
    A1 =( A1.subs(((xb,x[0] ),(yb, x[1]), (xc, x[2])))).evalf()
    w1 = (l - L.subs(((xb,x[0] ),(yb, x[1]), (xc, x[2])), dtype = float)).evalf()
    deltacap = (((A1.T * A1)).evalf().inv() * A1.T * w1).evalf()
    if np.all(abs(sp.matrix2numpy(deltacap)) < 0.00001):
        print(f"after {i+1} iteration the answer is = \n\n")
        display(x)
        break
    x = x + deltacap

after 2 iteration the answer is = 




Matrix([
[               499.988803707628],
[-0.03878509448861 + 500*sqrt(3)],
[                999.90483151484]])

In [9]:
# residual (سمت چپ معادله - سمت راست معادله)
w1

Matrix([
[3.79982676508883e-10],
[ 3.65064722192732e-9],
[8.14231913059351e-10]])

In [10]:
# adjustment(تصحیحات همگرا شده است)
deltacap

Matrix([
[2.50700580650041e-7],
[1.19413922884159e-6],
[5.59372776316701e-6]])

<img src="https://www.cs.utexas.edu/users/flame/laff/alaff-beta/images/Chapter04/FundamentalSpaces.png">

## Null space(Kernel) and left nullspace

In [11]:
A = np.array([[1, 3, -5, 1, 5],
             [1, 4, -7, 3, -2],
             [1, 5, -9, 5, -9],
             [1, 3, -6, 2, -1]])

In [12]:
np.linalg.matrix_rank(A) # rank of matrix 

3

In [16]:
Kernel = linalg.null_space(A)

In [17]:
Kernel # basis for nullspace(AX=0) | nullity : (dim = n - r) =>(5 - 3) = 2
#(2, n)

array([[ 0.36742309, -0.8771249 ],
       [ 0.67486077,  0.18355756],
       [ 0.22682793, -0.18256971],
       [-0.58300499, -0.40283879],
       [-0.13497215, -0.03671151]])

In [20]:
Ax = A @ Kernel * np.sign(Kernel[0, 0]) # sign ambiguity
Ax # zero(the difference is because of error)

array([[ 5.82867088e-16, -1.16573418e-15],
       [ 8.32667268e-16, -6.10622664e-16],
       [ 6.38378239e-16, -2.77555756e-16],
       [ 1.08246745e-15, -1.11022302e-16]])

In [22]:
# left kernel 
left_kernel = linalg.null_space(A.T)
left_kernel # basis for left kernel(A^TX=0)| dim(left kernel) = m - r
# m - r = 4 - 3 = 1 ==> (4, m)

array([[-4.08248290e-01],
       [ 8.16496581e-01],
       [-4.08248290e-01],
       [ 3.44169138e-15]])